In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('desafio-base1/evolucao.csv', parse_dates=['Data'], encoding='iso-8859-1', quotechar='"', delimiter='|')
df.shape

(1230, 6)

In [3]:
df.head(3).T

,0,1,2
Código,78,361,427
Data,2008-12-14 00:00:00,2009-03-27 00:00:00,2018-09-04 00:00:00
Procedimento,Consulta,Retorno,NaN
Tipo,HDA,HDA,Exame fisico
Medico,Teste 1,José,Teste 1
Evolução,Consequatur exercitationem adipisci. Labore qu...,Ducimus et error reprehenderit.\nAccusantium p...,Nesciunt quia ipsam quidem enim. Dolorum molli...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Código        1230 non-null   int64         
 1   Data          1230 non-null   datetime64[ns]
 2   Procedimento  1003 non-null   object        
 3   Tipo          1230 non-null   object        
 4   Medico        1230 non-null   object        
 5   Evolução      1230 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 57.8+ KB


In [5]:
# Colocando o nome das colunas do arquivo de acordo a documentação da iClinic
df = df.rename(
    columns = {
        "Código": "patient_id",
        "Data": "date",
        "Procedimento": "tab_name",
        "Tipo": "eventblock_name",
        "Evolução": "eventblock_text",
        "Medico": "physician_id",
    }
)

In [6]:
# Tratamento recurso 'patient_name'
# Este recurso precisa de informação externa para ser tratado, resumidamente, utilizaremos o índice em "patient_id" para localizar o nome do usuário no arquivo "patient.csv"
df_patient = pd.read_csv('desafio-base1-output/patient.csv', usecols = ["name"])

In [7]:
# Neste caso, como o 'patient_id' inicia com 0 e está ordenada é fácil de encontrar seu nome correspondente
index = df['patient_id'].values
df['patient_name'] = df_patient['name'][index].values

In [8]:
# Tratamento recurso 'physician_id'
# Este recurso também precisa de informação externa para ser tratado, resumidamente, buscaremos o nome na coluna "Medico" 
df_physician_names = pd.read_csv('desafio-base1-output/physician_names.csv')

In [9]:
# Neste caso, como o 'physician_names' está ordenada, podemos relacionar os indices de forma crescente
index_of_df_physician_names = pd.factorize(df_physician_names['physician_names'])[0].astype(int)
df['physician_id'] = df['physician_id'].replace(df_physician_names.values,index_of_df_physician_names)

In [10]:
#Adiconando as colunas ausentes
df['start_time'] = np.nan
df['end_time'] = np.nan

In [11]:
df = df.loc[:, ["patient_id","patient_name","physician_id","date","start_time","end_time","tab_name","eventblock_name","eventblock_text"]]

In [12]:
df.to_csv('desafio-base1-output/event_record.csv',index=False, encoding='utf-8')
